In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
import pandas as pd
import sys
import os
import numpy as np
import optuna

In [10]:

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = '/content/drive/Shareddrives/EEE405 Group Project/MagnetChallenge/data'
else:
    data_dir = 'C:/Users/moyin/Desktop/N87'

In [11]:
def load_material_data(material, data_dir=data_dir, first_row=0, last_row=None):
    """
    Load the material data from the data directory.
    :return: dict of pandas dataframes
    """
    material_dir = os.path.join(data_dir, material + "_cycle")
    if last_row is not None:
        nrows = last_row - first_row + 1
    else:
        nrows = None

    data_dict = {
        "Freq": pd.read_csv(os.path.join(material_dir, "Frequency[Hz].csv"), nrows=nrows, header=None, skiprows=first_row),
        "Temp": pd.read_csv(os.path.join(material_dir, "Temperature[C].csv"), nrows=nrows, header=None, skiprows=first_row),
        "B": pd.read_csv(os.path.join(material_dir, "B_waveform[T].csv"), header=None, nrows=nrows, skiprows=first_row),
        "H": pd.read_csv(os.path.join(material_dir, "H_waveform[Am-1].csv"), header=None, nrows=nrows, skiprows=first_row),
        "Loss": pd.read_csv(os.path.join(material_dir, "Volumetric_losses[Wm-3].csv"), header=None, nrows=nrows, names=["Loss"], skiprows=first_row),
    }

    df = pd.concat(data_dict.values(), axis=1, keys=data_dict.keys())
    return df


In [12]:
sine_dfs = load_material_data("N87", first_row=0, last_row=337)
triangle_dfs = load_material_data("N87", first_row=338, last_row=3649)
trap_dfs = load_material_data("N87", first_row=3650, last_row=10151)

In [15]:
def circular_shift_and_add_noise(waveform_df, num_shifts=5, noise_level=0.01):
    augmented_data = []

    for i in range(num_shifts):
        # Circular shift
        shifted_waveform = np.roll(waveform_df.values, shift=np.random.randint(len(waveform_df)), axis=0)

        # Add white noise
        noisy_waveform = shifted_waveform + noise_level * np.random.randn(*shifted_waveform.shape)

        augmented_data.append(noisy_waveform)

    augmented_data = np.concatenate(augmented_data, axis=0)

    # Create a DataFrame from the augmented data without changing column names
    augmented_df = pd.DataFrame(data=augmented_data, columns=waveform_df.columns)

    return augmented_df

In [18]:
sine_dfs = circular_shift_and_add_noise(sine_dfs, num_shifts=40, noise_level=0.01)
trap_dfs = circular_shift_and_add_noise(trap_dfs, num_shifts=2, noise_level=0.01)
triangle_dfs = circular_shift_and_add_noise(triangle_dfs, num_shifts=4, noise_level=0.01)

In [19]:
combined_df = pd.concat([sine_dfs, triangle_dfs, trap_dfs], ignore_index=True)

In [20]:
b_data = combined_df.loc[:, ('B', slice(None))].values

In [21]:
time_steps = 128
X_lstm_b = b_data.reshape(b_data.shape[0], time_steps, -1)

In [23]:
loss_data = combined_df.loc[:, ('Loss', 'Loss')].values.flatten()
y = np.log10(loss_data)

In [ ]:
X_lstm_b_tensor = torch.tensor(X_lstm_b, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # reshape for the output layer

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm_b_tensor, y_tensor, test_size=0.2, random_state=42)

# Convert the data into PyTorch DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [41]:
class CombinedModel(nn.Module):
    def __init__(self, input_size, hidden_layer1_size, hidden_layer2_size, hidden_layer3_size, dropout_rate):
        super(CombinedModel, self).__init__()
        # LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_layer1_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_layer1_size, hidden_layer2_size, batch_first=True)

        # FNN layers
        self.fc1 = nn.Linear(hidden_layer2_size, hidden_layer3_size)
        self.fc2 = nn.Linear(hidden_layer3_size, 15)
        self.fc3 = nn.Linear(15, 9)
        self.fc4 = nn.Linear(9, 1)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # LSTM layers
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x[:, -1, :].view(x.size(0), 1, -1))

        # FNN layers
        x = F.relu(self.fc1(x[:, -1, :]))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x


def relative_error(y_test, y_pred):
    avg_rpe = 100 * torch.mean(torch.abs((y_test - y_pred) / y_test))
    max_rpe = 100 * torch.max(torch.abs((y_test - y_pred) / y_test))
    return avg_rpe.item(), max_rpe.item()

# Define the training and evaluation loops
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    best_model = None
    best_avg_rpe = float('inf')  # Set to positive infinity for initialization
    best_max_rpe = float('inf')  # Set to positive infinity for initialization

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluation phase
        model.eval()
        with torch.no_grad():
            predictions, actuals = torch.tensor([]), torch.tensor([])
            for inputs, targets in test_loader:
                outputs = model(inputs)
                predictions = torch.cat((predictions, outputs), 0)
                actuals = torch.cat((actuals, targets), 0)

            # Calculate relative errors
            avg_rpe, max_rpe = relative_error(actuals, predictions)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Avg RPE: {avg_rpe:.2f}%, Max RPE: {max_rpe:.2f}%')

            # Check if this epoch has better relative errors
            if avg_rpe < best_avg_rpe:
                best_model = model.state_dict()  # Save the state_dict of the best model
                best_avg_rpe = avg_rpe
                best_max_rpe = max_rpe

    # Load the best model before returning
    model.load_state_dict(best_model)

    return model, best_avg_rpe, best_max_rpe


In [37]:
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer1_size = trial.suggest_int('hidden_layer1_size', 2, 50)
    hidden_layer2_size = trial.suggest_int('hidden_layer2_size', 2, 50)
    hidden_layer3_size = trial.suggest_int('hidden_layer3_size', 2, 50)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.9)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)

    
    model = CombinedModel(input_size=X_train.shape[2], hidden_layer1_size=hidden_layer1_size,
                          hidden_layer2_size=hidden_layer2_size, hidden_layer3_size=hidden_layer3_size,
                          dropout_rate=dropout_rate)

    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train and evaluate the model
    train_and_evaluate(model, train_loader, test_loader, criterion, optimizer)

    # Calculate mean squared error as the objective to minimize
    with torch.no_grad():
        predictions, actuals = torch.tensor([]), torch.tensor([])
        for inputs, targets in test_loader:
            outputs = model(inputs)
            predictions = torch.cat((predictions, outputs), 0)
            actuals = torch.cat((actuals, targets), 0)
        mse_loss = criterion(predictions, actuals)

    return mse_loss.item()




In [38]:

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2023-11-29 13:09:27,593] A new study created in memory with name: no-name-dcbc0756-7183-4125-b1ce-01dd532e4f90


Epoch [1/10], Loss: 0.6402, Avg RPE: 14.82%, Max RPE: 56.03%
Epoch [2/10], Loss: 0.5164, Avg RPE: 14.41%, Max RPE: 53.43%
Epoch [3/10], Loss: 0.1523, Avg RPE: 7.29%, Max RPE: 32.36%
Epoch [4/10], Loss: 0.1194, Avg RPE: 6.77%, Max RPE: 28.47%
Epoch [5/10], Loss: 0.1541, Avg RPE: 6.80%, Max RPE: 27.44%
Epoch [6/10], Loss: 0.1195, Avg RPE: 6.72%, Max RPE: 25.02%
Epoch [7/10], Loss: 0.1193, Avg RPE: 6.61%, Max RPE: 29.99%
Epoch [8/10], Loss: 0.1312, Avg RPE: 6.83%, Max RPE: 27.25%
Epoch [9/10], Loss: 0.1291, Avg RPE: 6.64%, Max RPE: 29.72%
Epoch [10/10], Loss: 0.1292, Avg RPE: 6.73%, Max RPE: 28.97%


[I 2023-11-29 13:12:36,920] Trial 0 finished with value: 0.12711727619171143 and parameters: {'hidden_layer1_size': 34, 'hidden_layer2_size': 24, 'hidden_layer3_size': 40, 'dropout_rate': 0.2224775459473375, 'learning_rate': 0.002408028917968372}. Best is trial 0 with value: 0.12711727619171143.


Epoch [1/10], Loss: 0.5782, Avg RPE: 14.65%, Max RPE: 54.99%
Epoch [2/10], Loss: 0.2905, Avg RPE: 10.57%, Max RPE: 40.79%
Epoch [3/10], Loss: 0.2524, Avg RPE: 10.17%, Max RPE: 42.35%
Epoch [4/10], Loss: 0.2558, Avg RPE: 8.63%, Max RPE: 36.75%
Epoch [5/10], Loss: 0.2860, Avg RPE: 9.59%, Max RPE: 37.42%
Epoch [6/10], Loss: 0.3400, Avg RPE: 9.90%, Max RPE: 41.36%
Epoch [7/10], Loss: 0.3329, Avg RPE: 9.89%, Max RPE: 40.32%
Epoch [8/10], Loss: 0.2749, Avg RPE: 9.71%, Max RPE: 37.95%
Epoch [9/10], Loss: 0.2623, Avg RPE: 9.61%, Max RPE: 37.77%
Epoch [10/10], Loss: 0.2413, Avg RPE: 9.18%, Max RPE: 28.99%


[I 2023-11-29 13:16:49,395] Trial 1 finished with value: 0.2996825873851776 and parameters: {'hidden_layer1_size': 42, 'hidden_layer2_size': 37, 'hidden_layer3_size': 33, 'dropout_rate': 0.1113874670770995, 'learning_rate': 0.018317757577226458}. Best is trial 0 with value: 0.12711727619171143.


...

Epoch [1/10], Loss: 0.5258, Avg RPE: 14.77%, Max RPE: 55.46%
Epoch [2/10], Loss: 0.5975, Avg RPE: 13.36%, Max RPE: 51.41%
Epoch [3/10], Loss: 0.2522, Avg RPE: 9.70%, Max RPE: 38.09%
Epoch [4/10], Loss: 0.3336, Avg RPE: 10.07%, Max RPE: 39.82%
Epoch [5/10], Loss: 0.2673, Avg RPE: 10.18%, Max RPE: 44.20%
Epoch [6/10], Loss: 0.2366, Avg RPE: 9.96%, Max RPE: 40.02%
Epoch [7/10], Loss: 0.2228, Avg RPE: 9.95%, Max RPE: 42.74%
Epoch [8/10], Loss: 0.2481, Avg RPE: 10.12%, Max RPE: 33.95%
Epoch [9/10], Loss: 0.3162, Avg RPE: 10.06%, Max RPE: 40.15%
Epoch [10/10], Loss: 0.2403, Avg RPE: 9.84%, Max RPE: 37.95%


[I 2023-11-29 13:36:30,530] Trial 8 finished with value: 0.29059067368507385 and parameters: {'hidden_layer1_size': 42, 'hidden_layer2_size': 30, 'hidden_layer3_size': 40, 'dropout_rate': 0.12362507402220671, 'learning_rate': 0.039070788804978274}. Best is trial 6 with value: 0.12709346413612366.


Epoch [1/10], Loss: 20.9816, Avg RPE: 95.89%, Max RPE: 97.12%
Epoch [2/10], Loss: 15.7237, Avg RPE: 79.62%, Max RPE: 85.69%
Epoch [3/10], Loss: 5.1929, Avg RPE: 45.32%, Max RPE: 61.61%
Epoch [4/10], Loss: 1.0542, Avg RPE: 16.38%, Max RPE: 37.46%
Epoch [5/10], Loss: 0.5695, Avg RPE: 14.58%, Max RPE: 52.94%
Epoch [6/10], Loss: 0.5300, Avg RPE: 14.74%, Max RPE: 55.11%
Epoch [7/10], Loss: 0.6706, Avg RPE: 14.75%, Max RPE: 55.27%
Epoch [8/10], Loss: 0.6967, Avg RPE: 14.73%, Max RPE: 54.99%
Epoch [9/10], Loss: 0.7010, Avg RPE: 14.76%, Max RPE: 55.40%
Epoch [10/10], Loss: 0.5286, Avg RPE: 14.73%, Max RPE: 55.06%


[I 2023-11-29 13:44:30,978] Trial 9 finished with value: 0.6000092625617981 and parameters: {'hidden_layer1_size': 50, 'hidden_layer2_size': 49, 'hidden_layer3_size': 2, 'dropout_rate': 0.41422936301466373, 'learning_rate': 0.0004203685286494101}. Best is trial 6 with value: 0.12709346413612366.


Number of finished trials: 10
Best trial:
Value:  0.12709346413612366
Params: 
    hidden_layer1_size: 12
    hidden_layer2_size: 46
    hidden_layer3_size: 32
    dropout_rate: 0.841073820164184
    learning_rate: 0.0016755379087784372


In [42]:
# Assuming you have already run the optimization and have the best trial
best_trial = study.best_trial

# Extract the best parameters
best_params = best_trial.params
hidden_layer1_size = best_params['hidden_layer1_size']
hidden_layer2_size = best_params['hidden_layer2_size']
hidden_layer3_size = best_params['hidden_layer3_size']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']

# Create an instance of CombinedModel with the best parameters
best_model = CombinedModel(input_size=X_train.shape[2], hidden_layer1_size=hidden_layer1_size,
                            hidden_layer2_size=hidden_layer2_size, hidden_layer3_size=hidden_layer3_size,
                            dropout_rate=dropout_rate)

# Define the loss function and the optimizer with the best learning rate
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=learning_rate)

# Train the model using the best parameters
trained_model, best_avg_rpe, best_max_rpe = train_and_evaluate(best_model, train_loader, test_loader, criterion, optimizer, num_epochs=50)


Epoch [1/50], Loss: 0.5703, Avg RPE: 13.86%, Max RPE: 52.61%
Epoch [2/50], Loss: 0.1554, Avg RPE: 7.81%, Max RPE: 31.27%
Epoch [3/50], Loss: 0.1300, Avg RPE: 7.13%, Max RPE: 26.96%
Epoch [4/50], Loss: 0.1448, Avg RPE: 6.99%, Max RPE: 26.33%
Epoch [5/50], Loss: 0.1381, Avg RPE: 6.83%, Max RPE: 26.15%
Epoch [6/50], Loss: 0.1381, Avg RPE: 6.67%, Max RPE: 28.08%
Epoch [7/50], Loss: 0.1121, Avg RPE: 6.62%, Max RPE: 31.21%
Epoch [8/50], Loss: 0.1409, Avg RPE: 6.65%, Max RPE: 29.83%
Epoch [9/50], Loss: 0.1563, Avg RPE: 6.65%, Max RPE: 25.57%
Epoch [10/50], Loss: 0.1324, Avg RPE: 6.72%, Max RPE: 29.22%
Epoch [11/50], Loss: 0.1445, Avg RPE: 6.83%, Max RPE: 30.95%
Epoch [12/50], Loss: 0.1095, Avg RPE: 6.56%, Max RPE: 27.91%
Epoch [13/50], Loss: 0.1328, Avg RPE: 6.58%, Max RPE: 29.98%
Epoch [14/50], Loss: 0.0959, Avg RPE: 6.55%, Max RPE: 29.77%
Epoch [15/50], Loss: 0.1102, Avg RPE: 6.61%, Max RPE: 29.27%
Epoch [16/50], Loss: 0.1219, Avg RPE: 6.65%, Max RPE: 30.52%
Epoch [17/50], Loss: 0.1404, Avg

In [44]:
torch.save(trained_model, 'C:/Users/moyin/Desktop/ML BASICS IN DEPLOYMENT TO FPGAS/EXTRA CREDIT/lstm.pth')

In [47]:
loaded_model = CombinedModel(input_size=X_train.shape[2], hidden_layer1_size=hidden_layer1_size,
                            hidden_layer2_size=hidden_layer2_size, hidden_layer3_size=hidden_layer3_size,
                            dropout_rate=dropout_rate)

torch.save(loaded_model.state_dict(), 'C:/Users/moyin/Desktop/ML BASICS IN DEPLOYMENT TO FPGAS/EXTRA CREDIT/lstm.pth')
loaded_model.eval()

CombinedModel(
  (lstm1): LSTM(8, 12, batch_first=True)
  (lstm2): LSTM(12, 46, batch_first=True)
  (fc1): Linear(in_features=46, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=9, bias=True)
  (fc4): Linear(in_features=9, out_features=1, bias=True)
  (dropout): Dropout(p=0.841073820164184, inplace=False)
)